In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from utils import compute_metrics, freeze_n_bottom_layers

### Dataset

In [2]:
dataset = load_dataset('emotion')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

### Model

In [3]:
model_chckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_chckpt)

/home/tgniazdo/miniconda3/envs/ner/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def tokenize_batch(batch, tokenizer=tokenizer):
    return tokenizer(batch['text'], padding=True, truncation=True)

dataset = dataset.map(tokenize_batch, batched=True, batch_size=None)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [5]:
dataset.set_format('pytorch')
train_ds = dataset['train']
val_ds = dataset['validation']

### Model training - with freezed n layers

In [6]:
batch_size = 4
logging_steps = len(train_ds) // batch_size
num_train_epochs = 3
learning_rate = 2e-5
weight_decay = 1e-5

for n in [6, 4, 2, -1]:
    tokenizer = AutoTokenizer.from_pretrained(model_chckpt)
    model = AutoModelForSequenceClassification.from_pretrained(model_chckpt, num_labels=6).to('cuda')
    model = freeze_n_bottom_layers(model, n, freeze_embeedings=True)

    training_args = TrainingArguments(
        output_dir=f'{model_chckpt}_emotions_freezed_{n}_layers',
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        evaluation_strategy='epoch',
        logging_steps=logging_steps,
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=val_ds,
    )

    print(f'Results for model with freezed {n} bottom layers.')
    trainer.train()
    print(trainer.evaluate())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Results for model with freezed 6 bottom layers.


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro,Acc
1,1.470400,1.381757,0.207064,0.505500,0.505500
2,1.351400,1.321477,0.210364,0.509500,0.509500
3,1.318200,1.306507,0.221535,0.515500,0.515500


{'eval_loss': 1.3065071105957031, 'eval_f1_macro': 0.22153481686870014, 'eval_f1_micro': 0.5155, 'eval_acc': 0.5155, 'eval_runtime': 5.8397, 'eval_samples_per_second': 342.481, 'eval_steps_per_second': 85.62, 'epoch': 3.0}


/home/tgniazdo/miniconda3/envs/ner/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

{'eval_loss': 0.269884318113327, 'eval_f1_macro': 0.9025633928250802, 'eval_f1_micro': 0.9315, 'eval_acc': 0.9315, 'eval_runtime': 5.8362, 'eval_samples_per_second': 342.691, 'eval_steps_per_second': 85.673, 'epoch': 3.0}


/home/tgniazdo/miniconda3/envs/ner/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Results for model with freezed -1 bottom layers.


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro,Acc
1,0.435500,0.285816,0.902415,0.929500,0.929500
2,0.187200,0.231340,0.913734,0.939000,0.939000
3,0.121300,0.253609,0.918432,0.940000,0.940000


{'eval_loss': 0.2536090910434723, 'eval_f1_macro': 0.918432422569797, 'eval_f1_micro': 0.94, 'eval_acc': 0.94, 'eval_runtime': 5.8674, 'eval_samples_per_second': 340.868, 'eval_steps_per_second': 85.217, 'epoch': 3.0}
